<a href="https://colab.research.google.com/github/pashok3d/RemarqueGPT/blob/main/RemarqueGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Building GPT from scratch and training it on all books of Erich Maria Remarque
"""

'\nBuilding GPT from scratch and training it on all books of Erich Maria Remarque\n'

In [2]:
!pip install tqdm -q
!pip install wandb -q
!pip install tiktoken -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.6 MB/s eta 0:00:00


In [4]:
!mkdir dataset
!mkdir model

mkdir: cannot create directory ‘dataset’: File exists
mkdir: cannot create directory ‘model’: File exists


In [6]:
import wandb
import tiktoken
import torch
import math
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import ConcatDataset, Dataset
from torch import nn
import math
from transformers import get_linear_schedule_with_warmup
from torch.nn.utils import clip_grad_norm_

In [14]:
from tokenizers import (
    Tokenizer,
    decoders,
    models,
    pre_tokenizers,
    trainers,
    processors,
    normalizers,
)

In [8]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
WINDOW_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 5
LR = 1e-4
EMBEDDING_DIM = 128
BLOCKS_NUM = 4
HEADS_NUM = 4
DROPOUT = 0.2
VOCAB_SIZE = 2024
MAX_GRAD_NORM = 1.0
VALIDATION_INTERVAL = 500  # Validate every 500 training steps

device = "cuda" if torch.cuda.is_available() else "cpu"

config = {
    "learning_rate": LR,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "window_size": WINDOW_SIZE,
    "embedding_dim": EMBEDDING_DIM,
    "blocks_num": BLOCKS_NUM,
    "heads_num": HEADS_NUM,
    "dropout": DROPOUT,
    "vocab_size": VOCAB_SIZE,
    "max_grad_norm": MAX_GRAD_NORM
}

In [10]:
run = wandb.init(project="remark-gpt", config=config)

wandb: Currently logged in as: crush_tarash. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [11]:
def generate_text(
    model,
    tokenizer: Tokenizer,
    prompt: str,
    device: str,
    window_size: int,
    max_tokens: int = 10,
    temperature: float = 1.0,
) -> str:
    """Generate text using the trained GPT model."""
    model.eval()
    context = tokenizer.encode(prompt).ids
    generated = list(context)

    with torch.no_grad():
        for _ in range(max_tokens):
            x = torch.tensor(context[-window_size:]).unsqueeze(0).to(device)
            logits, _ = model(x)
            logits = logits[0, -1, :] / temperature
            probs = torch.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()
            generated.append(next_token)
            context = generated

    return tokenizer.decode(generated)


class TextDataset(Dataset):
    def __init__(self, path, context_window_size, tokenizer: Tokenizer):
        # Load dataset
        with open(path, "r") as f:
            lines = f.readlines()

        text = "\n".join(lines)

        self.tokens = tokenizer.encode(text).ids

        self.x = []
        self.y = []
        for i in range(len(self.tokens) - context_window_size):
            self.x.append(self.tokens[i : i + context_window_size])
            self.y.append(self.tokens[i + 1 : i + context_window_size + 1])

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])


def get_datasets(paths, context_window_size, tokenizer: Tokenizer):
    datasets = []

    for path in paths:
        dataset = TextDataset(path, context_window_size, tokenizer)
        datasets.append(dataset)

    return datasets


In [23]:
# Prepare tokenizer
dataset_lines = []

dataset_paths = [
    "dataset/All_Quiet_on_the_Western_Front_1929_AST_978-5-17-105639-1.txt",
    "dataset/All_Quiet_on_the_Western_Front_1929_AST_978-5-17-137374-0.txt",
    "dataset/Station_at_the_Horizon_1928_AST_978-5-17-133322-5.txt",
    "dataset/The_Dream_Room_1920_AST_978-5-17-071518-3.txt",
    "dataset/The_Road_Back_1931.txt",
    "dataset/Tree_Comrades_1936_978-5-17-004252-4.txt",
    "dataset/Tree_Comrades_1936_978-5-17-056963-2.txt",
    "dataset/Tree_Comrades_1936_978-5-17-108545-2.txt"
]

for path in [ds_path.replace(".txt", "-train.txt") for ds_path in dataset_paths]:
    with open(path, "r") as f:
        dataset_lines.extend(f.readlines())

alphabet = list(set("".join(dataset_lines)))

tokenizer = Tokenizer(models.BPE())
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, initial_alphabet=alphabet)
tokenizer.normalizer = normalizers.Lowercase()
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.train_from_iterator(dataset_lines, trainer=trainer)

ds_list = get_datasets(
    [ds_path.replace(".txt", "-train.txt") for ds_path in dataset_paths],
    WINDOW_SIZE,
    tokenizer,
)

train_ds = ConcatDataset(ds_list)
train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

dev_dataset_paths = [ds_path.replace(".txt", "-dev.txt") for ds_path in dataset_paths]

dev_ds_list = get_datasets(dev_dataset_paths,
    WINDOW_SIZE,
    tokenizer,
)

names_with_dev_dataloaders = []

for i, dev_ds in enumerate(dev_ds_list):
    dev_dataloader = DataLoader(dev_ds, batch_size=BATCH_SIZE, shuffle=False)
    names_with_dev_dataloaders.append(
        (dev_dataset_paths[i].replace("dataset/", "").replace(".txt", ""), dev_dataloader)
    )

In [16]:
class AttentionHead(nn.Module):
    def __init__(self, embedding_dim, head_dim, dropout, max_len):
        super().__init__()

        self.Q = nn.Linear(embedding_dim, head_dim, bias=False)
        self.K = nn.Linear(embedding_dim, head_dim, bias=False)
        self.V = nn.Linear(embedding_dim, head_dim, bias=False)

        self.kv_softmax = nn.Softmax(dim=-1)
        self.attn_dropout = nn.Dropout(dropout)

        self.register_buffer("tril", torch.tril(torch.ones(max_len, max_len)))

    def forward(self, norm_inputs):

        _, T, _ = norm_inputs.shape

        q = self.Q(norm_inputs)
        k = self.K(norm_inputs)
        v = self.V(norm_inputs)

        attention_weights = (q @ k.transpose(-1, -2)) / math.sqrt(
            q.shape[-1]
        )  # shape: (B, T, T)
        attention_weights_masked = attention_weights.masked_fill(
            self.tril[:T, :T] == 0, -torch.inf
        )
        attention_scores = self.kv_softmax(attention_weights_masked)
        attention_scores = self.attn_dropout(attention_scores)

        return attention_scores @ v


class GPTBlock(nn.Module):
    def __init__(
        self, embedding_dim: int, max_len: int, dropout: float = 0.1, n_heads=2
    ):
        super().__init__()
        head_dim = embedding_dim // n_heads

        # Attention heads
        self.heads = nn.ModuleList(
            [
                AttentionHead(embedding_dim, head_dim, dropout, max_len)
                for _ in range(n_heads)
            ]
        )

        # Feedforward
        self.f1 = nn.Linear(embedding_dim, embedding_dim * 4)
        self.f_act = nn.ReLU()
        self.f2 = nn.Linear(embedding_dim * 4, embedding_dim)
        self.ff_dropout = nn.Dropout(dropout)

        self.ln1 = nn.LayerNorm(embedding_dim)
        self.ln2 = nn.LayerNorm(embedding_dim)

    def forward(self, inputs):
        norm_inputs = self.ln1(inputs)
        attn_out = torch.concat([h(norm_inputs) for h in self.heads], dim=-1)

        # Add residual connection
        x = inputs + attn_out

        norm_x = self.ln2(x)
        ff_out = self.ff_dropout(self.f2(self.f_act(self.f1(norm_x))))
        out = x + ff_out
        return out


class GPT(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        max_len: int,
        embedding_dim: int = 16,
        blocks_num: int = 4,
        n_heads: int = 2,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.emb = nn.Embedding(vocab_size, embedding_dim)
        self.pos = nn.Embedding(max_len, embedding_dim)
        self.blocks = nn.Sequential(
            *[
                GPTBlock(embedding_dim, max_len, dropout, n_heads)
                for _ in range(blocks_num)
            ]
        )
        self.proj = nn.Linear(embedding_dim, vocab_size)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, inputs, labels=None):
        _, T = inputs.shape
        embs = self.emb(inputs)
        pos_embs = self.pos(torch.arange(T, device=inputs.device))  # (T,C)
        blocks_output = self.blocks(embs + pos_embs)
        logits = self.proj(blocks_output)  # (B,T,vocab_size)
        if labels is not None:
            loss = self.loss_fn(logits.view(-1, self.vocab_size), labels.view(-1))
            return logits, loss
        else:
            return logits, None


In [17]:
model = GPT(vocab_size=tokenizer.get_vocab_size(), max_len=WINDOW_SIZE, embedding_dim=EMBEDDING_DIM, blocks_num=BLOCKS_NUM, n_heads=HEADS_NUM, dropout=DROPOUT)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

# Scheduler with warmup and linear decay
TOTAL_STEPS = EPOCHS * len(train_dataloader)
WARMUP_STEPS = int(0.05 * TOTAL_STEPS)  # 10% warmup

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=TOTAL_STEPS)

In [18]:
wandb.watch(model, log_freq=2500, log='all')

In [19]:
# model.train()
# epoch_loss = 0
# steps_n = 0
# with torch.no_grad():
#     for batch in tqdm(train_dataloader):
#         input, labels = batch[0].to(device), batch[1].to(device)
#         output, loss = model(input, labels)
#         epoch_loss += loss.item()
#         steps_n += 1
#     avg_loss = epoch_loss / steps_n
# expected_init_loss = -math.log(1 / tokenizer.get_vocab_size())
# print(f"initial train loss: {avg_loss:.3f}, with expected of {expected_init_loss:.3f}")

In [24]:
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    steps_n = 0
    total_steps = 0

    for batch in tqdm(train_dataloader):
        input, labels = batch[0].to(device), batch[1].to(device)
        output, loss = model(input, labels)
        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=MAX_GRAD_NORM)
        optimizer.step()
        scheduler.step()  # Update the learning rate
        optimizer.zero_grad()
        epoch_loss += loss.item()
        total_steps += 1
        steps_n += 1
        run.log({"train_loss": loss.item(), "lr": scheduler.get_last_lr()[0]})

    avg_loss = epoch_loss / steps_n
    print(f"epoch {epoch} train loss: {avg_loss:.3f}")

    if total_steps % VALIDATION_INTERVAL == 0:
        model.eval()
        with torch.no_grad():
            for ds_name, dev_dataloader in names_with_dev_dataloaders:
                val_steps_n = 0
                val_epoch_loss = 0
                for batch in tqdm(dev_dataloader):
                    input, labels = batch[0].to(device), batch[1].to(device)
                    output, loss = model(input, labels)
                    val_epoch_loss += loss.item()
                    val_steps_n += 1
                avg_val_loss = val_epoch_loss / val_steps_n
                print(f"val loss for {ds_name}: {avg_val_loss:.3f}")
                metric_name = f"avg_val_loss_{ds_name}"
                run.log({metric_name: avg_val_loss}, commit=False)

        run.log({"avg_train_loss": avg_loss})

100%|██████████| 17529/17529 [33:58<00:00,  8.60it/s]


epoch 0 train loss: 2.744


100%|██████████| 181/181 [00:07<00:00, 25.05it/s]


epoch 0 val loss for All_Quiet_on_the_Western_Front_1929_AST_978-5-17-105639-1-dev: 5.770


100%|██████████| 199/199 [00:07<00:00, 25.06it/s]


...

epoch 0 val loss for Tree_Comrades_1936_978-5-17-056963-2-dev: 5.419


100%|██████████| 378/378 [00:15<00:00, 24.99it/s]


epoch 0 val loss for Tree_Comrades_1936_978-5-17-108545-2-dev: 5.530


  0%|          | 32/17529 [00:03<35:05,  8.31it/s]


KeyboardInterrupt: 

In [25]:
torch.save(model.state_dict(), "model/gpt.pt")

In [26]:
artifact = wandb.Artifact("model", type="model")
artifact.add_file("model/gpt.pt")
run.log_artifact(artifact)

<Artifact model>

In [27]:
wandb.finish()

avg_train_loss,▁
avg_val_loss_All_Quiet_on_the_Western_Front_1929_AST_978-5-17-105639-1-dev,▁
avg_val_loss_All_Quiet_on_the_Western_Front_1929_AST_978-5-17-137374-0-dev,▁
avg_val_loss_Station_at_the_Horizon_1928_AST_978-5-17-133322-5-dev,▁
avg_val_loss_The_Dream_Room_1920_AST_978-5-17-071518-3-dev,▁
avg_val_loss_The_Road_Back_1931-dev,▁
avg_val_loss_Tree_Comrades_1936_978-5-17-004252-4-dev,▁
avg_val_loss_Tree_Comrades_1936_978-5-17-056963-2-dev,▁
avg_val_loss_Tree_Comrades_1936_978-5-17-108545-2-dev,▁
lr,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train_loss,█▇▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁


In [37]:
model.eval()
prompt = "Мы не в силах отвести глаза. Ферма, остатки леса,"
generated_text = generate_text(model, tokenizer, prompt, device, WINDOW_SIZE, max_tokens=500, temperature=0.3)

In [38]:
print(generated_text)

мы не в силах отвести глаза. ферма, остатки леса, холмы, эти подчинели в них несколько пастельных солдатских шинели, которые приготовлены из красной шинели, как своих зеленых зубах, литанные кровью; сапоги стонет крестьянами, растащит и жадно смотреть на него, как на лице играют светятся ярче, а там дороги, высокие и под огнем никто из нас не ощущает нас.

но мы не хотим верить этому. устоять, он питал под мышкой и, пролежал на кучку. потом по пути проехал обратно в мастерскую.

— идиот! — зарычал я на него и поспешил вернуться к блюменталю. — стоит вам проехаться разок на машине, и вы иначе станете относиться к цене. можете испытывать ее сколько пожелаете. если вам сейчас некогда, то я могу заехать за вами вечером, и мы совершим пробную поездку.

но вот из нас подводит элегантного, с каким удовольствием выговориться. но вилли, и альбертом показывает ровно по плечу:

— брайер!

— не знаю, не считай, что ли выговориться?

— нет, не получится. — она сняла растерянность. — зачем?

— за эт